# Poverty Rates for Cities Over 1M
- Joey Ashcroft Code Portion to be merged to master

In [51]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census
import gmaps

In [52]:
# Census & gmaps API Keys
from config import (api_key, gkey)

# Configure gmaps
gmaps.configure(api_key=gkey)

In [53]:
name = "?get=NAME,"
population = "B01003_001E"
poverty_count = "B17001_002E"
county = "for=county:*"

base_url = "https://api.census.gov/data/2017/acs/acs5"
query_url = base_url + name + population + "&" + poverty_count + "&" + county + "&key=" + api_key
query_url

'https://api.census.gov/data/2017/acs/acs5?get=NAME,B01003_001E&B17001_002E&for=county:*&key=e4802ddb92bf7b297f5f10de4176b9d5341c10c8'

In [54]:
census_response = requests.get(query_url)
census_response = census_response.json()

# Convert to DataFrame
poverty_data = pd.DataFrame(census_response)

#replace header with row 1
new_header = poverty_data.iloc[0] #grab the first row for the header
poverty_data = poverty_data[1:] #take the data less the header row
poverty_data.columns = new_header #set the header row as the df header

# Column Reordering
poverty_data = poverty_data.rename(columns={"B01003_001E": "Population",
                                            "B17001_002E": "Poverty Count",
                                            "NAME": "Name", 
                                            "state": "State",
                                            "county":"County"})

# Add in Poverty Rate (Poverty Count / Population)
poverty_data["Poverty Rate"] = 100 * poverty_data["Poverty Count"].astype(int) / poverty_data["Population"].astype(int)

poverty_data.head()

,Name,Population,Poverty Count,State,County,Poverty Rate
1,"Corozal Municipio, Puerto Rico",34933,19146,72,047,54.807775
2,"Maunabo Municipio, Puerto Rico",11297,5945,72,095,52.624591
3,"Peñuelas Municipio, Puerto Rico",21661,12379,72,111,57.148793
4,"Ponce Municipio, Puerto Rico",148863,75187,72,113,50.507514
5,"San Sebastián Municipio, Puerto Rico",38970,21472,72,131,55.098794


In [55]:
poverty_data["County"] = poverty_data["Name"].str.split(pat=",", expand=True)[0]
poverty_data["State"] = poverty_data["Name"].str.split(pat=",", expand=True)[1]
# Parse Final DataFrame
poverty_data = poverty_data[["County","State","Poverty Rate"]]
poverty_data.head()

,County,State,Poverty Rate
1,Corozal Municipio,Puerto Rico,54.807775
2,Maunabo Municipio,Puerto Rico,52.624591
3,Peñuelas Municipio,Puerto Rico,57.148793
4,Ponce Municipio,Puerto Rico,50.507514
5,San Sebastián Municipio,Puerto Rico,55.098794
